Exemple de la [documentation](https://spark.apache.org/docs/latest/ml-features#onehotencoderestimator)

In [2]:
df = spark.createDataFrame([
    ("couleur 1", "forme 3"),
    ("couleur 2", "forme 1"),
    ("couleur 1", "forme 1"),
    ("couleur 3", "forme 2"),
    ("couleur 1", "forme 1"),
    ("couleur 2", "forme 3")
], ["categorycarac1", "categorycarac2"])
display(df)

categorycarac1,categorycarac2
couleur 1,forme 3
couleur 2,forme 1
couleur 1,forme 1
couleur 3,forme 2
couleur 1,forme 1
couleur 2,forme 3


In [3]:
from pyspark.ml.feature import StringIndexer

# On passe les caractères en numériques avec StringIndexer
indexer = StringIndexer(inputCol="categorycarac1", outputCol="categoryIndex1").fit(df)
df = indexer.transform(df)
indexer = StringIndexer(inputCol="categorycarac2", outputCol="categoryIndex2").fit(df)
df = indexer.transform(df)
display(df)

categorycarac1,categorycarac2,categoryIndex1,categoryIndex2
couleur 1,forme 3,0.0,1.0
couleur 2,forme 1,1.0,0.0
couleur 1,forme 1,0.0,0.0
couleur 3,forme 2,2.0,2.0
couleur 1,forme 1,0.0,0.0
couleur 2,forme 3,1.0,1.0


In [4]:
# On passe chaque vecteur numérique en vecteur de booléens
# le OneHotEncoder estimator crée n-1 valeurs binaires regroupées dans un même vecteur
# avec n = nombre de valeurs du vecteur
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["categoryIndex1", "categoryIndex2"],
                                 outputCols=["categoryVec1", "categoryVec2"])
model = encoder.fit(df)
encoded = model.transform(df)
encoded.show()

+--------------+--------------+--------------+--------------+-------------+-------------+
categorycarac1|categorycarac2|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+--------------+--------------+-------------+-------------+
 couleur 1| forme 3| 0.0| 1.0|(2,[0],[1.0])|(2,[1],[1.0])|
 couleur 2| forme 1| 1.0| 0.0|(2,[1],[1.0])|(2,[0],[1.0])|
 couleur 1| forme 1| 0.0| 0.0|(2,[0],[1.0])|(2,[0],[1.0])|
 couleur 3| forme 2| 2.0| 2.0| (2,[],[])| (2,[],[])|
 couleur 1| forme 1| 0.0| 0.0|(2,[0],[1.0])|(2,[0],[1.0])|
 couleur 2| forme 3| 1.0| 1.0|(2,[1],[1.0])|(2,[1],[1.0])|
+--------------+--------------+--------------+--------------+-------------+-------------+

In [5]:
print(encoded.select("categoryVec1").collect())

[Row(categoryVec1=SparseVector(2, {0: 1.0})), Row(categoryVec1=SparseVector(2, {1: 1.0})), Row(categoryVec1=SparseVector(2, {0: 1.0})), Row(categoryVec1=SparseVector(2, {})), Row(categoryVec1=SparseVector(2, {0: 1.0})), Row(categoryVec1=SparseVector(2, {1: 1.0}))]

In [6]:
# on peut faire un VectorAssembler sur tout ce beau monde 
# note : aucun problème pour faire un vectorAssembler avec des 
# colonnes vecteurs et non vecteurs
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["categoryVec1", "categoryVec2"],outputCol="features")
encoded_2 =  assembler.transform(encoded)
encoded_2.show()

+--------------+--------------+--------------+--------------+-------------+-------------+-----------------+
categorycarac1|categorycarac2|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2| features|
+--------------+--------------+--------------+--------------+-------------+-------------+-----------------+
 couleur 1| forme 3| 0.0| 1.0|(2,[0],[1.0])|(2,[1],[1.0])|[1.0,0.0,0.0,1.0]|
 couleur 2| forme 1| 1.0| 0.0|(2,[1],[1.0])|(2,[0],[1.0])|[0.0,1.0,1.0,0.0]|
 couleur 1| forme 1| 0.0| 0.0|(2,[0],[1.0])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0]|
 couleur 3| forme 2| 2.0| 2.0| (2,[],[])| (2,[],[])| (4,[],[])|
 couleur 1| forme 1| 0.0| 0.0|(2,[0],[1.0])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0]|
 couleur 2| forme 3| 1.0| 1.0|(2,[1],[1.0])|(2,[1],[1.0])|[0.0,1.0,0.0,1.0]|
+--------------+--------------+--------------+--------------+-------------+-------------+-----------------+